In [1]:
import time
from tqdm import tqdm

In [2]:
import tensorflow as tf

2024-04-09 07:46:51.218369: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-09 07:46:51.774906: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from tensorflow.keras.models import load_model

In [4]:
import numpy as np

In [5]:
model = load_model('surogate.h5')

2024-04-09 07:46:53.716970: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9604 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2024-04-09 07:46:53.717472: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9621 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


In [6]:
# 特定频率
def loss_freq(s_para):
    # 能量
    E = tf.square(s_para)
    E11 = E[:,0] + E[:,1]
    E21 = E[:,2] + E[:,3]
    E31 = E[:,4] + E[:,5]
    E41 = E[:,6] + E[:,7]
    P21 = tf.math.atan2(s_para[:,3], s_para[:,2])
    P31 = tf.math.atan2(s_para[:,5], s_para[:,4])

    # 压制
    loss1 = E11 - E21 - E31 + E41
    # 比例
    loss2 = tf.abs(E21 / (E31 + E21) - 0.7)
    # phase
    loss3 = tf.abs(P21 - P31 - np.pi / 4)
    # 能量守恒
    loss4 = tf.abs(tf.reduce_sum(E, axis=1) - 1)
    
    return loss1 + loss2 + loss3 + loss4

In [7]:
def dgn_obj(s1, s2, s3):
    # var = tf.reduce_sum(tf.nn.moments(tf.stack([s1, s2, s3], axis=1), [1])[1][:, 2: 6], axis=-1)
    # return tf.reduce_max([loss_freq(s1), loss_freq(s2), loss_freq(s3)], axis=0)
    return loss_freq(s1) + loss_freq(s2) + loss_freq(s3)

In [8]:
# 参数数量，迭代次数
num_node_epochs = 100

In [9]:
# 物理边界检查函数
def check(GeomData):
    inva_place1 = tf.where(tf.logical_or(GeomData[:,:8] < 1, GeomData[:,:8] > 5))
    GeomData = tf.tensor_scatter_nd_update(GeomData, [inva_place1], [np.random.uniform(mmin[inva_place1[:,1]], mmax[inva_place1[:,1]], (inva_place1.shape[0]))])
    
    inva_place2 = tf.where(tf.logical_or(GeomData[:,8:] < 4, GeomData[:,8:] > 100)) + [0, 8]
    GeomData = tf.tensor_scatter_nd_update(GeomData, [inva_place2], [np.random.uniform(mmin[inva_place2[:,1]], mmax[inva_place2[:,1]], (inva_place2.shape[0]))])
    
    return tf.Variable(GeomData)

In [10]:
# 物理边界
mmin = np.array([1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4])
mmax = np.array([5, 5, 5, 5, 5, 5, 5, 5, 100, 100, 100, 100])

In [11]:
# 初始数据
freq1 = np.ones((40000, 1), dtype=np.float32) * 2.4
freq2 = np.ones((40000, 1), dtype=np.float32) * 2.5
freq3 = np.ones((40000, 1), dtype=np.float32) * 2.6

In [12]:
def myOpti(num_nodes):
    geom = tf.Variable(np.random.uniform(mmin, mmax, (num_nodes, 12)), dtype=tf.float32)
    bestLoss = 10
    bestGeom = 0
    # 创建优化器
    opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.01)
    # 优化过程
    t0 = time.time()
    for i in tqdm(range(num_node_epochs)):
        with tf.GradientTape(watch_accessed_variables=False) as tape:
            tape.watch(geom)
            y_pred1 = model(tf.concat([freq1[:num_nodes], geom], axis=1))
            y_pred2 = model(tf.concat([freq2[:num_nodes], geom], axis=1))
            y_pred3 = model(tf.concat([freq3[:num_nodes], geom], axis=1))
            loss = dgn_obj(y_pred1, y_pred2, y_pred3)
        minLoss = tf.reduce_min(loss).numpy()
        minIndex = tf.argmin(loss).numpy()
        minGeom = geom[minIndex].numpy()
        grads = tape.gradient(loss, geom)
        opt.apply_gradients(grads_and_vars=zip([grads], [geom]))
        geom = check(geom)
        if minLoss < bestLoss:
            bestLoss = minLoss
            bestGeom = minGeom
            # print(minIndex)
            # print(i, bestLoss)
            # print(" ".join(map(str, bestGeom)))
            # print()
    return (time.time() - t0) / num_node_epochs
    # logs.append({'N-sample': num_nodes, 'Avg-time': (time.time() - t0) / num_node_epochs})

In [13]:
logs = []

In [14]:
for i in range(200, 20001, 200):
    logs.append(myOpti(i))

100%|█████████████████████████████████████████| 100/100 [00:08<00:00, 12.29it/s]


In [15]:
for log in logs:
    print(log)

0.051979973316192626
0.04683565378189087
0.04688331842422486
0.046717319488525394
0.0471727204322815
0.04682141780853272
0.04682702541351318
0.046873884201049806
0.04713475704193115
0.04698659420013428
0.04736770391464233
0.047444028854370116
0.04750128746032715
0.04689696788787842
0.046791951656341556
0.048000376224517825
0.047316665649414065
0.04731592655181885
0.04684596061706543
0.047959444522857664
0.0468750262260437
0.047744736671447754
0.047564969062805176
0.04807984590530395
0.047798590660095217
0.0486633563041687
0.04955560445785522
0.04997167110443115
0.0503980278968811
0.050346677303314206
0.051202995777130125
0.0521389365196228
0.052252368927001955
0.05307734251022339
0.05333032608032227
0.05347726106643677
0.05467345237731933
0.054898269176483154
0.05614584684371948
0.05630230903625488
0.05610100746154785
0.054944453239440916
0.056773979663848874
0.05494408130645752
0.056063225269317625
0.05440303087234497
0.056628541946411134
0.05609769582748413
0.056275949478149415
0.057